In [3]:
import pandas as pd
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [4]:
look = pd.read_csv('C:/Users/LG/Desktop/데이터분석/프로젝트/최종프로젝트/3차/looker_(datetime).csv')

C:\Users\LG\AppData\Local\Temp\ipykernel_1784\1191939609.py:1: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  look = pd.read_csv('C:/Users/LG/Desktop/데이터분석/프로젝트/최종프로젝트/3차/looker_(datetime).csv')


In [5]:
def convert_type(df):
    for col in df.columns:
        if '_at' in col:    # 시간 데이터
            df = df.assign(**{col:pd.to_datetime(df[col], format='ISO8601')})
            print('[+] {0:<30} >>> datetime'.format(col))
        elif '_id' in col:  # id 컬럼 -> obj형으로
            df = df.assign(**{col:df[col].astype('object')})
            print('[+] {0:<30} >>> object'.format(col))
    return df

In [6]:
look = convert_type(look)

[+] user_id                        >>> object
[+] account_created_at             >>> datetime
[+] order_id                       >>> object
[+] order_items_id                 >>> object
[+] order_created_at               >>> datetime
[+] shipped_at                     >>> datetime
[+] delivered_at                   >>> datetime
[+] returned_at                    >>> datetime
[+] event_id                       >>> object
[+] session_id                     >>> object
[+] login_user_id                  >>> object
[+] sess_created_at                >>> datetime
[+] inventory_item_id              >>> object
[+] product_id                     >>> object
[+] inventory_created_at           >>> datetime
[+] inventory_sold_at              >>> datetime


In [7]:
df = look.copy()

LTR 계산하기 : 매출 : 고객 수
- 특정기간 내에 가입한 고객의 수
- 특정 기간 내에 첫 구매를 한 고객의 수
- 특정 기간 내에 서비스 이용을 사작한 고객 수

1. 2024 RFM 분류별 LTR VS 
2. 기존 이탈고객의 LTR, 신규 이탈 고객의 LTR

In [8]:
df.head()

,user_id,age,gender,account_state,account_city,account_country,latitude,longitude,account_traffic_source,account_created_at,...,sess_state,sess_city,inventory_item_id,product_id,category,brand,product_name,inventory_created_at,inventory_sold_at,cost
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,Chungcheongnam-do,Asan City,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,North Carolina,Cary,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,Nouvelle-Aquitaine,Le Gua,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,Florida,New Port Richey East,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,Pennsylvania,York,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN


In [9]:
#1.최근일자 구하기
maxdate = df['order_created_at'].max()+dt.timedelta(days=1)
maxdate

Timestamp('2024-01-22 17:57:14.434458+0000', tz='UTC')

In [10]:
# 2.고객별 rfm등급 구하기
df = df.dropna(subset=['user_id'])
rfm_df = df.groupby(['user_id']).agg({'order_created_at':lambda x :(maxdate-max(x)).days,
                                        'order_items_id':'count',
                                        'sale_price':'sum'}).rename(columns={'order_created_at':'Recency','order_items_id':'Frequency','sale_price':'Monetary'})

In [11]:
rfm_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 96874 entries, 1.0 to 100000.0
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Recency    76918 non-null  float64
 1   Frequency  96874 non-null  int64  
 2   Monetary   96874 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 3.0 MB


In [12]:
def segments(rfm_df):
    if (rfm_df['Recency']<94) & (rfm_df['Frequency']>=4) & (rfm_df['Monetary']>490):
        val = 'important vip'
    elif ((rfm_df['Recency']>=94)&(rfm_df['Recency']<365)&(rfm_df['Monetary']>400))|((rfm_df['Recency']<94)&(rfm_df['Frequency']<4)&(rfm_df['Monetary']>400))|((rfm_df['Recency']<94)&(rfm_df['Frequency']>=4)&(rfm_df['Monetary']>400)&(rfm_df['Monetary']<=490)):
        val = 'normal vip'
    elif (rfm_df['Recency']>=365)&(rfm_df['Monetary']>400):
        val = 'dormant vip'
    elif (rfm_df['Recency']<150)&(rfm_df['Frequency']>=4)&(rfm_df['Monetary']>110)&(rfm_df['Monetary']<=400):
        val = 'important standard'
    elif ((rfm_df['Recency']>=150)&(rfm_df['Recency']<365)&(rfm_df['Monetary']<=400))|((rfm_df['Recency']<150)&(rfm_df['Frequency']<4)&(rfm_df['Monetary']<=400))|((rfm_df['Recency']<150)&(rfm_df['Frequency']>=4)&(rfm_df['Monetary']<=110)):
        val = 'normal standard'
    elif (rfm_df['Recency']>=365)&(rfm_df['Monetary']<=400):
        val = 'dormant standard'
    else:
        val = 'na'
    return val

In [13]:
rfm_df['segment'] = rfm_df.apply(lambda x : segments(x), axis =1)

In [23]:
rfm_df = rfm_df.reset_index()
rfm_df

,index,user_id,Recency,Frequency,Monetary,segment
0,0,1.0,551.0,3,159.990000,dormant standard
1,1,2.0,701.0,1,22.000000,dormant standard
2,2,3.0,167.0,4,338.230000,normal standard
3,3,4.0,461.0,1,148.000000,dormant standard
4,4,5.0,455.0,3,65.650000,dormant standard
...,...,...,...,...,...,...
96869,96869,99996.0,313.0,4,123.870001,normal standard
96870,96870,99997.0,NaN,0,0.000000,na
96871,96871,99998.0,184.0,3,116.990000,normal standard
96872,96872,99999.0,30.0,2,67.980000,normal standard


In [25]:
df = pd.merge(df,rfm_df[['segment','user_id']], on='user_id', how ='inner')

In [71]:
# 3. 각 등급별 23년도 ltr과 24년도 ltr구하기
df24 = df[df['sess_created_at'].dt.year == 2024]
df23 = df[df['sess_created_at'].dt.year == 2023]

In [73]:
df23.head(1)

,user_id,age,gender,account_state,account_city,account_country,latitude,longitude,account_traffic_source,account_created_at,...,sess_city,inventory_item_id,product_id,category,brand,product_name,inventory_created_at,inventory_sold_at,cost,segment
0,76411.0,54.0,F,Heilongjiang,Wenzhou,China,45.713922,126.769299,Search,2023-06-30,...,Wenzhou,375421.0,14215.0,Accessories,Aquapro,Radiant Navy Blue Pleated Turban Hat Head Cove...,2023-11-03,NaT,2.60352,normal standard


In [74]:
segment23_total_sale = df23.groupby(['segment'],as_index=False)['sale_price'].sum()
segment23_total_sale

,segment,sale_price
0,dormant standard,9.433802e+04
1,dormant vip,9.297410e+03
2,important standard,5.317080e+05
3,important vip,1.163771e+05
4,normal standard,2.870035e+06
5,normal vip,4.560904e+05


In [75]:
segment23_user = df23.groupby(['segment'],as_index=False)['user_id'].nunique()
segment23_user

,segment,user_id
0,dormant standard,1434
1,dormant vip,35
2,important standard,3141
3,important vip,247
4,normal standard,36244
5,normal vip,1256


In [76]:
# 23년도 고객별 LTR계산
segment23 = pd.merge(segment23_total_sale,segment23_user,on='segment',how='inner')
segment23['LTR'] = round(segment23['sale_price']/segment23['user_id'],2)
segment23.sort_values('segment',inplace=True)
segment23

,segment,sale_price,user_id,LTR
0,dormant standard,9.433802e+04,1434,65.79
1,dormant vip,9.297410e+03,35,265.64
2,important standard,5.317080e+05,3141,169.28
3,important vip,1.163771e+05,247,471.16
4,normal standard,2.870035e+06,36244,79.19
5,normal vip,4.560904e+05,1256,363.13


In [80]:
# 3. 24년도 이탈고객 분류하기
# 이탈고객 불러오기
churn24 = pd.read_csv('C:/Users/LG/Desktop/데이터분석/프로젝트/최종프로젝트/3차/추가/churnid_24.csv')
churn24

,id,Target
0,11.0,1
1,13.0,1
2,22.0,1
3,26.0,1
4,45.0,1
...,...,...
6838,99964.0,1
6839,99968.0,1
6840,99970.0,1
6841,99971.0,1


In [84]:
# 24년도 이탈고객의 등급별 LTR계산
# df24 = pd.merge(df24,rfm_df[['segment','user_id']], on='user_id', how ='inner')
churn24_df = pd.merge(churn24, df24, left_on='id',right_on='user_id', how='inner')
# df_24['user_id'].nunique() 6843

In [86]:
churn24_total_sale = df24.groupby(['segment'],as_index=False)['sale_price'].sum()
churn24_user = df24.groupby(['segment'],as_index=False)['user_id'].nunique()
churn_segment24 = pd.merge(churn24_total_sale,churn24_user,on='segment',how='inner')
churn_segment24['LTR'] = round(churn_segment24['sale_price']/churn_segment24['user_id'],2)
churn_segment24.sort_values('segment',inplace=True)
churn_segment24

,segment,sale_price,user_id,LTR
0,important standard,125824.110175,901,139.65
1,important vip,44533.250017,123,362.06
2,normal standard,425413.140595,5639,75.44
3,normal vip,60005.400000,180,333.36


In [ ]:
# 24년도 고객별 LTR계산(이탈+방문 예측)
# df24 = pd.merge(df24,rfm_df[['segment','user_id']], on='user_id', how ='inner')
# segment24_total_sale = df24.groupby(['segment'],as_index=False)['sale_price'].sum()
# segment24_user = df24.groupby(['segment'],as_index=False)['user_id'].nunique()
# segment24 = pd.merge(segment24_total_sale,segment24_user,on='segment',how='inner')
# segment24['LTR'] = round(segment24['sale_price']/segment24['user_id'],2)
# segment24.sort_values('segment',inplace=True)
# segment24

,segment,sale_price,user_id,LTR
0,important standard,125824.110175,901,139.65
1,important vip,44533.250017,123,362.06
2,normal standard,425413.140595,5639,75.44
3,normal vip,60005.400000,180,333.36


In [91]:
LTR = pd.merge(segment23,churn_segment24,on='segment',suffixes=('_23','_24'))
LTR['rate'] = round(LTR['LTR_24']/LTR['LTR_23'],2)
LTR

,segment,sale_price_23,user_id_23,LTR_23,sale_price_24,user_id_24,LTR_24,rate
0,important standard,5.317080e+05,3141,169.28,125824.110175,901,139.65,0.82
1,important vip,1.163771e+05,247,471.16,44533.250017,123,362.06,0.77
2,normal standard,2.870035e+06,36244,79.19,425413.140595,5639,75.44,0.95
3,normal vip,4.560904e+05,1256,363.13,60005.400000,180,333.36,0.92
